In [ ]:
import openai
import os
import pandas as pd

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
df = pd.read_csv("sales_data_sample.csv")

In [ ]:
df.head()

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import text

In [ ]:
# Create a temporary database in RAM
temp_db = create_engine("sqlite:///:memory:", echo=True)

In [ ]:
# Add df to temporary database as sql table
data = df.to_sql(name="sales",con=temp_db)

In [ ]:
# Run an sql query with the connection to this sql database.
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT sum(SALES) FROM sales"))

In [ ]:
result.all()

In [ ]:
# Now, I need to create a prompt to deliver OpenAI the information about the table I have. I create a function for that.
def create_table(df):
    prompt = """### sqlite SQL table, with its properties:
    #
    # Sales({})
    #  
    """.format(",".join(str(col) for col in df.columns))
    return prompt

In [ ]:
print(create_table(df))

In [ ]:
# For a non-technical person, create a prompt input
def prompt_input():
    npl_text = input("Enter the info you want:")
    return npl_text

In [ ]:
prompt_input()

In [ ]:
# Now, it is time to combine of them all
def combine_prompts(df, query):
    table_def = create_table(df)
    query_string = f"###A query to answer: {query}\nSELECT"
    return table_def+query_string

In [ ]:
nlp_text = prompt_input()
combine_prompts(df,nlp_text)
# So in the end, this one will be send to GPT

# Muhtemelen bu Davinci ile text completion yapilarak denenecek. Sonra aynisini ChatGPT ile cagiracagim.